初级训练器lgb最优参数

In [ ]:
params_lgb = {
    'learning_rate':0.1,
    'max_bin':150,
    'num_leaves':32,    
    'max_depth':11,
    
    'reg_alpha':0.1,
    'reg_lambda':0.2,   
     
    'objective':'multiclass',
    'n_estimators':300,
    #'class_weight':weight
}

初级训练器catbost最优参数

In [2]:
params_cat = {
         'n_estimators': 30000,
         'loss_function': 'MAE',
         'eval_metric':'MAE',
         'learning_rate': 0.015,
         'depth': 5,
         'use_best_model': True,
         'bootstrap_type': 'Bernoulli',
         'reg_lambda': 3
    #'class_weight':weight
}

初级训练器xgb最有参数

In [3]:
params_xgb={
        'eta':0.2,                        #特征权重，取值范围0~1，通常最后设置eta为0.01~0.2
        'max_depth':7,                #树的深度，通常取值3-10，过大容易过拟合，过小欠拟合
        'min_child_weight':3,             #最小样本的权重，调大参数可以繁殖过拟合
        'gamma':0.2,                      #控制是否后剪枝，越大越保守，一般0.1、 0.2的样子
        'subsample':0.8,                  #随机取样比例
        'colsample_bytree':0.8 ,          #默认为1，取值0~1，对特征随机采集比例
        'reg_lambda':0.2,
        'reg_alpha':0.4,
        'learning_rate':0.01,
        'n_estimators':1200,
        'booster':'gbtree',               #迭代树
        'objective':'multiclass',    #逻辑回归，输出为概率
        'nthread':6,                      #设置最大的进程量，若不设置则会使用全部资源
        'scale_pos_weight':1,             #默认为0,1可以处理类别不平衡
        'lambda':1,                       #默认为1，用于L2平滑处理项，避免模型过拟合
        'seed':1234,                      #随机树种子
        'silent':1,                       #0表示输出结果
        'eval_metric':'auc'    #评分指标

    }

次级训练器xgboost最优参数

In [6]:
param_xgb_second = {
        'learning_rate':0.1,
        'max_depth':3,
        'num_leaves':32,    
        'min_child_weight':3,             
        'gamma':0.2,                    
        'subsample':0.8,                 
        'colsample_bytree':0.8 ,          
        'reg_lambda':0.2,
        'reg_alpha':0.4,
}

导入包

In [159]:
from xgboost.sklearn import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin, clone
import numpy as np
from sklearn import  metrics 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

加载数据

In [ ]:
Data = np.load('data.npy')
x, y = Data[0], Data[1]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4)  

stacking 类

In [150]:
class StackingAveragedModels1(BaseEstimator, ClassifierMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds

    # 我们将原来的模型clone出来，并且进行实现fit功能
    def fit(self, X, y, label):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)

        #对于每个模型，使用交叉验证的方法来训练初级学习器，并且得到次级训练集
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)*3))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                instance.fit(X[train_index], y[train_index])
                self.base_models_[i].append(instance)
                y_pred = instance.predict_proba(X[holdout_index])
                print(y_pred.shape)
                out_of_fold_predictions[holdout_index, i*label:(i+1)*label] = y_pred
               
             
        # 使用次级训练集来训练次级学习
        print(len(out_of_fold_predictions[0]))
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    #在上面的fit方法当中，我们已经将我们训练出来的初级学习器和次级学习器保存下来了
    #predict的时候只需要用这些学习器构造我们的次级预测数据集并且进行预测就可以了
    def predict(self, X):
        
        meta_features = np.column_stack(np.dstack(model.predict_proba(X) for model in base_models).mean(axis=-1) for base_models in self.base_models_ )
        return self.meta_model_.predict(meta_features)

        # meta_features = np.column_stack([
        #     np.column_dtack([model.predict_proba(X) for model in base_models]).mean(axis=2)
        #     for base_models in self.base_models_ ])
        # return self.meta_model_.predict(meta_features)

加载模型

In [151]:
xgb = XGBClassifier(**params_xgb)
cat = CatBoostClassifier(**params_cat)
lgb = lgb.LGBMClassifier(**params_lgb)
xgb_second =CatBoostClassifier(**params_xgb_second)
base_models = [xgb, cat, lgb]
meta_model = xgb_second
model = StackingAveragedModels1(base_models, meta_model)

训练模型


In [156]:
model.fit(x_train, y_train, 3)
preds_class = mon.predict(x_test)[:,0]
accuracy = accuracy_score(y_test, preds_class)
print ("Accuracy:", accuracy)

(18, 3)
(18, 3)
(18, 3)
(18, 3)
(18, 3)
(18, 3)
(18, 3)
(18, 3)
(18, 3)
(18, 3)
(18, 3)
(18, 3)
(18, 3)
(18, 3)
(18, 3)
9
Accuracy: 0.9


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
